In [175]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [176]:
pickle_file = './res/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [177]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [178]:
# 定义初始化weight函数
# 依然使用截断的truncated_normal分布来初始化权重
# 加一个L2的loss防止过拟合
def variable_with_weight_loss(shape, stddev, w1):
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if w1 is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var), w1, name="weight_loss")
        tf.add_to_collection('losses', weight_loss)
    return var

In [179]:
# 计算loss
# 把softmax与cross_entropy联合在一起使用
# 然后计算均值
# 然后添加到整体的losses中，得到最终的loss（里面包含两个全连接层的L2）
def loss_union(logits, labels):
    
#     tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=labels)
    cross_entropy_mean = tf.reduce_mean(cross_entropy,
        name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)

    return tf.add_n(tf.get_collection('losses'), name='total_loss')


In [241]:
# http://blog.sina.com.cn/s/blog_1768c4a370102ye1y.html
# https://www.ritchieng.com/machine-learning/deep-learning/tensorflow/deep-neural-nets/

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  """
  定义全连接不同层的神经元个数，每层连接数的数量，是2整数倍时，效果较好。
  """
#   L = 200
#   M = 100
#   N = 60
#   O = 30

  L = 1024
  M = 512
  N = 256
  O = 32


  """
  这里的两个参数，W0 和 S0，当他们是相差10倍的时候，结果相对较好。这一组参数， 从 0.001, 0.01  -->  0.009, 0.09 中， 
  0.005, 0.05这一组的效果相对较好。
  DROPOUT0 从 0.3 --> 0.955,效果是递增的。其中在0.3时表现相当差，应该是对参数隐藏得过头了。
  """

  W0 = 0.005
  S0 = 0.05
  DROPOUT0 = 0.955
    
  W1 = variable_with_weight_loss([image_size * image_size, L], stddev=S0, w1=W0 )
  B1 = tf.Variable(tf.ones([L])/10)

  W2 = variable_with_weight_loss([L, M], stddev=S0, w1=W0)
  B2 = tf.Variable(tf.ones([M])/10)

  W3 = variable_with_weight_loss([M, N], stddev=S0, w1=W0)
  B3 = tf.Variable(tf.ones([N])/10)

  W4 = variable_with_weight_loss([N, O], stddev=S0, w1=W0)
  B4 = tf.Variable(tf.ones([O])/10)

  W5 = variable_with_weight_loss([O, num_labels], stddev=1/O, w1=0.0)
  B5 = tf.Variable(tf.zeros([num_labels]))
    
  """
  用relu作为激活函数，然后传递到下一层，最后一层的数量是输出分类的数量。
  """  
  keep_prob = tf.placeholder("float")
  XX_Train = tf.reshape(tf_train_dataset,[-1, image_size * image_size])
  Y11 = tf.nn.relu(tf.matmul(XX_Train, W1) + B1)
  Y1 = tf.nn.dropout(Y11, keep_prob)

  Y21 = tf.nn.relu(tf.matmul(Y1, W2) + B2)  
  Y2 = tf.nn.dropout(Y21, keep_prob)

  Y31 = tf.nn.relu(tf.matmul(Y2, W3) + B3)
  Y3 = tf.nn.dropout(Y31, keep_prob)

  Y41 = tf.nn.relu(tf.matmul(Y3, W4) + B4)    
  Y4 = tf.nn.dropout(Y41, keep_prob)

  logits = tf.matmul(Y4, W5) + B5    

  XX_valid = tf.reshape(tf_valid_dataset,[-1, image_size * image_size])
  YV1 = tf.nn.relu(tf.matmul(XX_valid, W1) + B1)
  YV2 = tf.nn.relu(tf.matmul(YV1, W2) + B2)    
  YV3 = tf.nn.relu(tf.matmul(YV2, W3) + B3)    
  YV4 = tf.nn.relu(tf.matmul(YV3, W4) + B4)    
  logits_valid = tf.matmul(YV4, W5) + B5

  XX_test = tf.reshape(tf_test_dataset,[-1, image_size * image_size])
  YT1 = tf.nn.relu(tf.matmul(XX_test, W1) + B1)
  YT2 = tf.nn.relu(tf.matmul(YT1, W2) + B2)    
  YT3 = tf.nn.relu(tf.matmul(YT2, W3) + B3)    
  YT4 = tf.nn.relu(tf.matmul(YT3, W4) + B4)    
  logits_test = tf.matmul(YT4, W5) + B5    
        
  # Training computation.
  
#   logits = tf.matmul(tf_train_dataset, weights) + biases
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
#   loss = loss_union(logits=logits, labels=tf_train_labels)


    

# https://www.cnblogs.com/gengyi/p/9898960.html
# learning_rate - 初始学习率
# global_step - 用于衰减计算的全局步骤。 一定不为负数。喂入一次 BACTH_SIZE 计为一次 global_step
# decay_steps - 衰减速度，一定不能为负数，每间隔decay_steps次更新一次learning_rate值
# decay_rate - 衰减系数，衰减速率，其具体意义参看函数计算方程(对应α^t中的α)。
# staircase - 若 ‘ True ’ ，则学习率衰减呈 ‘ 离散间隔 ’ （discrete intervals），
#             具体地讲，`global_step / decay_steps`是整数除法，衰减学习率（ the decayed learning rate ）遵循阶梯函数；
#             若为 ’ False ‘ ，则更新学习率的值是一个连续的过程，每步都会更新学习率。
# 返回值：

# 与初始学习率 learning_rate 相同的标量 Tensor 
#  优点：

# 训练伊始可以使用较大学习率，以快速得到比较优的解。
# 后期通过逐步衰减后的学习率进行迭代训练，以使模型在训练后期更加稳定。            

  start_learning_rate = 0.85
  learning_rate = 0.1
  decay_rate = 0.98
  global_steps = 10500
  decay_steps = 100

  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(start_learning_rate, 
                                             global_step, 
                                             decay_steps, 
                                             decay_rate, 
                                             staircase=False)
    
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
  # Optimizer.
#   optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(logits_valid)
  test_prediction = tf.nn.softmax(logits_test)


In [243]:
num_steps = 10501
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : DROPOUT0 }
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.300513
Minibatch accuracy: 8.6%
Validation accuracy: 15.4%
Minibatch loss at step 500: 0.513846
Minibatch accuracy: 85.2%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 0.400647
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 1500: 0.526859
Minibatch accuracy: 86.7%
Validation accuracy: 88.0%
Minibatch loss at step 2000: 0.263145
Minibatch accuracy: 90.6%
Validation accuracy: 88.6%
Minibatch loss at step 2500: 0.306616
Minibatch accuracy: 90.6%
Validation accuracy: 88.9%
Minibatch loss at step 3000: 0.284250
Minibatch accuracy: 90.6%
Validation accuracy: 89.2%
Minibatch loss at step 3500: 0.383810
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 4000: 0.251778
Minibatch accuracy: 92.2%
Validation accuracy: 90.2%
Minibatch loss at step 4500: 0.322761
Minibatch accuracy: 91.4%
Validation accuracy: 90.1%
Minibatch loss at step 5000: 0.219882
Minibatch accuracy: 93.0%
Validation accuracy